In [32]:
BUCKET_NAME='test-xwh01'
OBS_BASE_PATH=BUCKET_NAME

from modelarts.session import Session
session = Session()

session.download_data(bucket_path="test-xwh01/test-machine/ratings.dat", path="./ratings.dat")
session.download_data(bucket_path="test-xwh01/test-machine/users.dat", path="./users.dat")
session.download_data(bucket_path="test-xwh01/test-machine/movies.dat", path="./movies.dat")
# import same usefull libraries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_rating = pd.read_csv('./ratings.dat')

Successfully download file test-xwh01/test-machine/ratings.dat from OBS to local ./ratings.dat
Successfully download file test-xwh01/test-machine/users.dat from OBS to local ./users.dat
Successfully download file test-xwh01/test-machine/movies.dat from OBS to local ./movies.dat


In [33]:
movie_rating_cols = ['user_id',  'movie_id', 'rating', 'unix_tiemstamp'] 
movie_rating = pd.read_csv('./ratings.dat',sep = '::', names = movie_rating_cols, parse_dates = True)
movie_rating.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,user_id,movie_id,rating,unix_tiemstamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [34]:
movie_rating.nunique()

user_id             6040
movie_id            3706
rating                 5
unix_tiemstamp    458455
dtype: int64

In [35]:
user=pd.read_csv('./users.dat',sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'])
user.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [36]:
user.shape

(6040, 5)

In [37]:
movie=pd.read_csv('./movies.dat',sep='::',names=['MovieID','Title','Genres'])
movie.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [38]:
movie.shape

(3883, 3)

In [39]:
movie_rating.drop("unix_tiemstamp",inplace = True, axis = 1)
movie_rating.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [40]:
# 拆分训练集与数据集
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(movie_rating,test_size = 0.3)

In [41]:
# 创建用户电影矩阵
movie_rating_matrix = train_data.pivot_table(index = ['movie_id'], columns = ['user_id'],
                                              values = 'rating').reset_index(drop = True)
movie_rating_matrix.fillna(0, inplace = True)
cmu = movie_rating_matrix
cmu.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# 利用余弦算法获得电影相似度矩阵
movie_similarity = 1 - pairwise_distances(movie_rating_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity,0)
# 将相似度填回相似矩阵
movie_rating_matrix = pd.DataFrame(movie_similarity)
movie_rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3651,3652,3653,3654,3655,3656,3657,3658,3659,3660
0,0.000000,0.286099,0.179378,0.124400,0.174252,0.244015,0.226745,0.093309,0.076674,0.265774,...,0.065817,0.033796,0.057487,0.055524,0.044357,0.222511,0.151217,0.074272,0.029720,0.121912
1,0.286099,0.000000,0.169686,0.111154,0.160023,0.178302,0.184375,0.151541,0.118376,0.278667,...,0.046783,0.015909,0.040592,0.057824,0.041629,0.131322,0.096495,0.023573,0.017988,0.094726
2,0.179378,0.169686,0.000000,0.146308,0.219716,0.125578,0.211908,0.043387,0.098226,0.143244,...,0.036037,0.053350,0.044550,0.059055,0.046268,0.130483,0.065976,0.068387,0.014477,0.058603
3,0.124400,0.111154,0.146308,0.000000,0.197334,0.086123,0.152467,0.018630,0.036884,0.076474,...,0.023272,0.000000,0.000000,0.025154,0.000000,0.059552,0.058340,0.015739,0.000000,0.048843
4,0.174252,0.160023,0.219716,0.197334,0.000000,0.104386,0.216170,0.059983,0.121302,0.149136,...,0.024739,0.053106,0.028743,0.016451,0.015881,0.118426,0.065396,0.043004,0.006005,0.070192


In [43]:
# 获得TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = movie_rating_matrix.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.286099
2,3,Grumpier Old Men (1995),Comedy|Romance,0.179378
3,4,Waiting to Exhale (1995),Comedy|Drama,0.124400
4,5,Father of the Bride Part II (1995),Comedy,0.174252


In [44]:
# 测试集合构造
rating_test_matrix = test_data.pivot_table(index = ['movie_id'],columns = ['user_id'],
                                values = 'rating').reset_index(drop = True)
rating_test_matrix.fillna(0,inplace = True)
cmu_test = rating_test_matrix
cmu_test.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# 测试集构造物品相似矩阵
movie_similarity_test = 1 - pairwise_distances(rating_test_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity_test,0)
rating_matrix_test = pd.DataFrame(movie_similarity_test)
rating_matrix_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3536,3537,3538,3539,3540,3541,3542,3543,3544,3545
0,0.000000,0.084830,0.073170,0.046575,0.083105,0.137148,0.091851,0.029540,0.027415,0.119499,...,0.012279,0.000000,0.00000,0.020664,0.007140,0.080491,0.048852,0.030696,0.014429,0.056760
1,0.084830,0.000000,0.072727,0.088558,0.095236,0.067797,0.068570,0.089028,0.032568,0.082022,...,0.021424,0.000000,0.04786,0.037877,0.033221,0.056991,0.048544,0.034946,0.000000,0.058921
2,0.073170,0.072727,0.000000,0.052208,0.106051,0.036138,0.100033,0.043481,0.046845,0.078000,...,0.007952,0.000000,0.01938,0.024333,0.004624,0.031928,0.030526,0.000000,0.000000,0.021090
3,0.046575,0.088558,0.052208,0.000000,0.089853,0.019098,0.092444,0.072206,0.011950,0.043144,...,0.078605,0.000000,0.00000,0.000000,0.000000,0.063120,0.028917,0.000000,0.000000,0.027795
4,0.083105,0.095236,0.106051,0.089853,0.000000,0.026252,0.045842,0.056717,0.030798,0.081270,...,0.000000,0.099244,0.00000,0.019838,0.000000,0.066412,0.050118,0.000000,0.000000,0.037411


In [47]:
# 测试集TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = rating_matrix_test.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.084830
2,3,Grumpier Old Men (1995),Comedy|Romance,0.073170
3,4,Waiting to Exhale (1995),Comedy|Drama,0.046575
4,5,Father of the Bride Part II (1995),Comedy,0.083105


In [50]:
# 训练集预测，求得RMSE
# 将dataframe形式的用户物品相似矩阵转为numpy 数组
rating_matrix_array = movie_rating_matrix.values
user_item_pre = rating_matrix_array.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis = 1)])

pre_flatten = user_item_pre
rating_matrix_flatten = rating_matrix_array
from sklearn.metrics import mean_squared_error
from math import sqrt
error_train = sqrt(mean_squared_error(pre_flatten, rating_matrix_flatten))
print(error_train)


0.045651659725245874


In [52]:
# 测试集预测，求得RMSE
rating_test_matrix_array = rating_matrix_test.values
user_item_test_pre = rating_test_matrix_array.dot(movie_similarity_test) / np.array([np.abs(movie_similarity_test).sum(axis = 1)])

test_pre_flatten = user_item_test_pre
test_rating_matrix_flatten = rating_test_matrix_array
error_test = sqrt(mean_squared_error(test_pre_flatten, test_rating_matrix_flatten))
print(error_test)

0.028747288332633493
